# Wprowadzenie 2/3

Nasze zajęcia na dzisiaj:
    
1. Dane Treningowe i Testowe,
2. Przygotowanie algorytmów klasyfikacja i etykiety,
3. Algorytmy Supervised Learning
4. Przykłady algorytmów i ich zastosowania

## Praktyczny ML

### Drivetrain Approach

![](https://raw.githubusercontent.com/fastai/fastbook/823b69e00aa1e1c1a45fe88bd346f11e8f89c1ff//images/drivetrain-approach.png)

Na podstawie [fastbook](https://github.com/fastai/fastbook/blob/master/02_production.ipynb)

Tak jak każdy projekt ML powienien mieć jasno określony cel, który pozwoli na jego efektywne zaimplementowanie.

## Tracer-bullet approach

Czyli jak najszybciej staramy się uruchomić nasz pipeline dla algorytmu, żeby:

1. Mieć punkt odniesienia dla zmian,
2. Poznać lepiej domenę,
3. Analizując przykłady danych dla których nasz algorytm się najgoszej sprawdza (największy *loss*), często wskazuje na problemy z danymi lub naszym zrozumieniem problemu.

## Dane

```{mermaid}
flowchart LR
  TD(Training\nData) -- trenujemy\n/uczymy --> TM[Trenowanie\nModelu]
  VD(Validation\nData) -- czy idziemy\nw dobra strone? --> TM
  
  TM --> M(Model)
  
  M --> SM[Sprawdzenie\nModelu]
  
  
  TeD(Test\nData) -- jak sprawdza się\nna nieznanych danych --> SM
  
  SM --> Metryki(Metryki)
```

## Typy Danych

Mamy:

1. **Dane Treningowe**, czasami mówimy o danych treningowych jako dane uczące i validacyjne, czasami narzędzia, które będziemy używać będą wewnętrznie obsługiwać podział dane do trenowania i walidacji.
2. **Dane Walidacyjne**, pozwalają nam dać feedback dla algorytmu uczącego, a propos jakości wyników.
3. **Dane Testowe**, dane, które nasz algorytm nie widział i nie miał okazji się nauczyć.

## Klasyfikacja i Etykiety

Po angielsku `label`, dodatkowa informacja wyjaśniająca co wiersz danych, zestaw pikseli, lub fragment piosenki oznacza. Etykiety są tym co chcemy przewidzieć na podstawie danych, które ją reprezentują. Celem algorytmów klasyfikacji jest właśne znalezienie sposobu rozpoznania etykiety na podstawie danych wejściowych.

```{mermaid}

flowchart LR
  Dane_1(x11,x12, x13, x14) --> Etykieta_1("Label 1 (y)")
  Dane_2(x21,x22, x23, x14) --> Etykieta_1
  
  Dane_3(x31,x32, x33, x34) --> Etykieta_2("Label 2 (y)")
```

**Algorytmy klasyfikacji**

## Funkcje w popularnych biblioteki ML / DS

### scikit-learn

Na przykład bibliotek [scikit-learn](https://scikit-learn.org/stable/) dostarcza nam funkcję `train_test_split` ([dokumentacja](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).

Jeśli zaleźy nam na losowym wybraniu zbiorów testowych i treningowych wystarczy:

```python
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
)
```

Jeśli chcemy się upewnić, że mamy zarówno w danych testowych oraz trenujacych reprezentacje dla każdej klasy, jeśli przygotowujemy dane do wytrenowania algorytmu do klasyfikacji:

```python
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=22, stratify=y
)
```

Musimy upewnić się, że zarówno X i y mają reprezentantów każdej z klasy (etykiety), inaczej albo nasz model nie będzie w stanie obsłużyć danej klasy, albo nie będziemy w stanie pomierzyć efektywności naszego algorytmu.

## fastai

W przypadku [fastai](https://docs.fast.ai/data.load.html), [DataBlock](https://docs.fast.ai/data.block.html) ([dokumentacja na temat transformacji danych](https://docs.fast.ai/data.transforms.html)) daje nam logikę do wyznaczenia `X`, `y` oraz rozdzielenia danych na validacyjne i do trenowania.

```python
DataBlock(
    blocks=(TextBlock.from_df("full_text", vocab=dls_lm.vocab), CategoryBlock), # funkcje do transformacji danych
    get_x=ColReader("text"), # co jest X
    get_y=ColReader("category"), # co jest y
    splitter=ColSplitter("is_valid")) # ktore wiersze beda train, a ktore valid
```



## Praca z danymi, transformacje i feature engineering

Zauważ: zazwyczaj w pierwszej iteracji, na mniejszej ilości danych, wyprobowyjemy jeden z algorytmów, który wiemy, że może dać nam dobre wyniki. Daje nam to punkt odniesienia, dla poprawy danych czy też wypróbowywania różnych algorytmów.

### Problemy

```{mermaid}

flowchart LR
   PBW(Brakujace\nwartości) --> Problemy(Problemy z danymi)
   PBR(Dziwne\nreprezentacje pustych wartości) --> Problemy
   PBO(Wartości\nodstające) --> Problemy
   PBZ(Zakresy wartości\ndanych wejściowych) --> Problemy
   PB(Brak wystarczająco\nreprezentantów dla danej klasy) --> Problemy
   
```

W świecie idealnym:

- dane są kompletne, a w przypadku problemów klasyfikacji, mamy przykłady dla każdej z etykiet/klas;
- nie mamy braków, pustych wartości, dziwnych znaczników, np., NULL, null, ... itp;
- dane są w podobnych zakresach, np., 1-10, bez outlinerów.

taaaa... 

### Zagrożenia w produkcji

Warto zdawać sobie sprawę, model w produkcji wymaga od nas szczególnej uwagi, ze względu na zagrożenia:

```{mermaid}

flowchart LR
   Bias(Bias) --> Z("Zagrożenia")
   MR("model rot") --> Z
   FL("Feedback loops") --> Z
   OD("out-of-domain data") --> Z
```


Zagrożenia w produkcji:

- bias - nasz dataset może być tendencyjny, np., trenowanie rozpoznawania anglieskiego poza szkocją czy NZ
- model rot - nasze dane trenujące nie odpowiadają rzeczywistości w której model operatuje
- feedback loops - sugerujemy się obecnymi wynikami modelu w doborze danych do dotrenowania
- out-of-domain data - dane, których nie widzieliśmy nigdy wcześniej

### Rollout


Dlatego rollout nowego modelu powinien być uważnie monitorowany:

![](https://raw.githubusercontent.com/fastai/fastbook/823b69e00aa1e1c1a45fe88bd346f11e8f89c1ff//images/att_00061.png){width=40%}

## Prace z danymi

Na szczęście większość bibliotek dostarcza nam gotowe funkcję, większość bibliotek pozwala nam budować pipeline-y, które łączą z sobą wiele funkcjonalności.

Zobaczmy na przykładzie... 

xxxxxxx panda z danymi kat i numerycznymi


In [23]:
# explicit

In [24]:
# transofrmator

In [25]:
# uzupełnianie pustych

In [26]:
# encoder

In [27]:
# wolne!


Na szczęście większość bibliotek dostarcza nam gotowe funkcje od których możemy zacząć:

In [19]:
## Przykłady kodu

In [20]:
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=22, stratify=y
)

NameError: name 'X' is not defined

## Przykłady algorytmów

![](https://upload.wikimedia.org/wikipedia/commons/9/90/Task-guidance.png){width=30%}

Źródło [wikipedia](https://en.wikipedia.org/wiki/Supervised_learning)

### Supervised learning


### Przykłady i zastosowanie

In [ ]:
# x